In [1]:
import pandas as pd
import numpy as np
import nltk
import re

In [2]:
# !pip install nltk

In [3]:
# 5+6
# !pip install spacy

In [4]:
import nltk
nltk.__version__

'3.8.1'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
import string
import spacy



/workspaces/twitter_sentiment_analysis/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# df = pd.read_csv("training.csv", encoding="ISO-8859-1")
df = pd.read_csv("sentiment140_with_location.csv", header=None, names=["target", "unique_id", "date", "flag", "user_id", "text","location"], encoding="ISO-8859-1")
df.sample(10)

,target,unique_id,date,flag,user_id,text,location
130855,0,1835394931,Mon May 18 05:53:38 PDT 2009,NO_QUERY,shidazzle,remembered my login details lol!! not been up ...,Dominican Republic
268455,0,1989408996,Mon Jun 01 01:25:57 PDT 2009,NO_QUERY,Tripl3A,@noturgirlfri3nd damn so is that a no??,Tanzania
1475942,4,2065995953,Sun Jun 07 09:41:28 PDT 2009,NO_QUERY,Jade_XD,Facebook + Twitter + Multiply + Ym + Youtube +...,Canada
606553,0,2222637361,Thu Jun 18 07:14:59 PDT 2009,NO_QUERY,shinyhappydan,@8tracks I can't upload any tracks it tries f...,Jamaica
869817,4,1678218785,Sat May 02 05:57:43 PDT 2009,NO_QUERY,xTwiggy,@helloxxtaylor thats EXACTLY the type of person,Nigeria
1279411,4,2001535253,Tue Jun 02 01:12:21 PDT 2009,NO_QUERY,AmazingBecause,@ChicagoMonica hope u have a wonderful day &am...,France
1015860,4,1881619244,Fri May 22 05:08:39 PDT 2009,NO_QUERY,jessroisin,"oh, it's 09 a.m and I'm waiting my students.. ...",Thailand
204812,0,1972711337,Sat May 30 09:44:34 PDT 2009,NO_QUERY,backseat_poet,photography job commenced. it was hot outside ...,Argentina
1369867,4,2050903936,Fri Jun 05 19:30:15 PDT 2009,NO_QUERY,marxee,"@fed_memb yeah, outside Luzon...may outing kam...",India
857223,4,1573894518,Tue Apr 21 03:21:37 PDT 2009,NO_QUERY,Eamando,chillaxin doing some coursework........not qui...,Canada


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   target     1600000 non-null  int64 
 1   unique_id  1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   flag       1600000 non-null  object
 4   user_id    1600000 non-null  object
 5   text       1600000 non-null  object
 6   location   1600000 non-null  object
dtypes: int64(2), object(5)
memory usage: 85.4+ MB


In [8]:
df.shape

(1600000, 7)

In [9]:
df.drop(['unique_id','date','flag','user_id'], axis = 1, inplace = True) 

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   target    1600000 non-null  int64 
 1   text      1600000 non-null  object
 2   location  1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB


In [11]:
df.sample(10)

,target,text,location
315362,0,grad practice,Australia
1202090,4,there can be miracLes..when you beLieve..,Chile
1146001,4,To all my Tweeps...I'm officially moving to @b...,Cuba
1547501,4,Vote for @mileycyrus in the teen choice awards...,France
1081260,4,@goofyfootkush ooh yay thanks i'll give you t...,Colombia
822440,4,@Iconic88 You're most welcome.,Canada
64166,0,@zombiesohno yeah dude. seriously. there were ...,Chile
430594,0,"About to clean, clean, clean, make some breakf...",Colombia
1361530,4,"@YankeeGirl20 that sounds like a plan, have a ...",Ethiopia
897005,4,Gettn ready to watch the TOUGH LOVE finale!ahh...,Peru


In [12]:
df.rename(columns={'target': 'sentiment', 'text': 'tweet'}, inplace = True)

In [13]:
df.sample(10)

,sentiment,tweet,location
1524236,4,This William Beckett video is going really slo...,China
1331421,4,Welcome @iamenina Bye! Good Night!,Spain
577015,0,@shawnajalynn aww I hope everythin is alright...,Mexico
1540245,4,BAAALLROOM BLITZ So nostalgic right now.,Chile
727424,0,sore throat,South Africa
305566,0,my phones completely DUNZO,China
592370,0,Is stressing over marketing!,Egypt
1591368,4,good morning everyone!,Australia
122548,0,hopes she does well in her exams!!! Especially...,Peru
469503,0,This has been the worst week ever... I just wa...,Tanzania


In [14]:

df = df.dropna()
df.sample(10)

,sentiment,tweet,location
64421,0,Made it back safe &amp; sound from the swine c...,Indonesia
336120,0,Going to Sac tmwr morning w/my uncle &amp; aun...,Jamaica
1152361,4,Did a video chat with the aunt and uncle we're...,Germany
830870,4,Susan Boyle's 1999 cover of Cry Me A River: h...,Japan
981752,4,@kobe213 thanks doll,Argentina
682962,0,@BendyyStrawz because u dumped me,France
945555,4,@Wendys_World that's a really cool place. I w...,Kenya
270085,0,"@BlowhornOz @TvFanUk Im good, Hows you? At col...",Nigeria
1340941,4,@James_Phelps I didn't know you play! That's s...,Indonesia
776225,0,God I hope he's ok... He didn't deserve that!....,France


In [15]:
print(df.info())
print("The Shape Of The Dataset Is :",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   tweet      1600000 non-null  object
 2   location   1600000 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.6+ MB
None
The Shape Of The Dataset Is : (1600000, 3)


In [16]:
df['sentiment'] = df['sentiment'].replace({0: 'negative', 4: 'positive'})

In [17]:
df.sample(5)

,sentiment,tweet,location
218198,negative,"@Mom_20 oh yeah, that too! i love fun jewelry ...",Thailand
31915,negative,School and work are killing me I have no time...,Argentina
377292,negative,@makeupmanda aww lol! It would have cheered me...,Thailand
1248929,positive,@amylovesoliver I'm being buttinski. You haaaa...,New Zealand
1340146,positive,"If u have a Google Wave account, feel free to ...",Argentina


In [18]:
# Remove URLs, special characters, punctuation, and numbers from tweet column
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-Z\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[0-9]+', '', x))


In [19]:
print(df.sample(10))
print('\033[1m' + 'The information of the dataset is :' + '\033[0m')
print(df.info())
print('\033[1m' + 'The Shape Of The Dataset Is :' + '\033[0m',df.shape)

        sentiment                                              tweet  \
312509   negative  punctuation LizSra I think some people have it...   
230307   negative  Morning all  Slow start today Bed making to do...   
1069120  positive                           With hunter and miranda    
414372   negative  AlaskaCook Hey there I kiiiiinda hate you righ...   
181475   negative                     home tired  and is kinda upset   
1452818  positive  I wonder what it would be like to write a file...   
7542     negative                apdesch because he doesnt like you    
42023    negative      what kind of boyfriend ignores you this much    
1102047  positive  Send pics  Anoop Pillow case to me  SEJlvcedu ...   
1597453  positive  Is happy bout life I just ate a small bag of g...   

          location  
312509       China  
230307   Indonesia  
1069120  Argentina  
414372        Peru  
181475      Brazil  
1452818     Brazil  
7542       Germany  
42023         Cuba  
1102047      Egypt

In [20]:
df['tweet'] = df['tweet'].str.lower()

In [21]:
df.sample(10)

,sentiment,tweet,location
64734,negative,failingwords swore emily lol because my footba...,Kenya
128377,negative,in studio i wanna go home fell like i may up c...,India
911616,positive,adventureaddict awesome congrats on completing...,Canada
366388,negative,kenzieance banquet tonight really excited to s...,Colombia
923068,positive,back from shopping managed to spend quite a l...,South Korea
5477,negative,would have been delighted by this snow a few d...,Ecuador
1324843,positive,brokencyde track heard last night sound kinda ...,Russia
1294372,positive,tommcfly yeah youve just changed your pic nice...,Jamaica
661409,negative,pumpkincore,Jamaica
1107841,positive,twlady doesnt he dogs have it figured out un...,Peru


In [22]:
# stop_words = set(stopwords.words('english'))
# df['tweet'] = df['tweet'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x))
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

stop_words = set(stopwords.words('english'))
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

stemmer = PorterStemmer()
df['tweet'] = df['tweet'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

In [25]:
df.sample(10)

,sentiment,tweet,location
497482,negative,miss long hair nyay,New Zealand
1559843,positive,geezushaberdash moribathek im bwb trust want f...,Dominican Republic
1131295,positive,realiz quit liber drive sunday morn everyon el...,USA
791831,negative,still doin uni work got take lil boy doctor pr...,Peru
1024326,positive,pragmatist vanillari layer nice top skin scrub...,Argentina
286984,negative,thought go friend famili passeng air franc flight,France
279411,negative,wait atampt call tell opteman chang make chang...,Ecuador
77433,negative,kick bloodi bookcas toe throb like mofo,Egypt
463026,negative,twitterrif suffer twitpocolyps,Cuba
163356,negative,gleed need tickettttt gah ya youv got reason h...,Australia
